In [ ]:
import json
import os
import json
from selenium import webdriver
import time
import re
from stravalib.client import Client


def initialize_json(json_path):
    if not os.path.exists(json_path):
        
        print('This is the first time you use this. Please login to https://www.strava.com/settings/api and create a new application. ')
        
        client_id = input(f"Please enter your client id: ")
        client_secret = input(f"Please enter your client secret: ")
        
        client = Client()
        request_scope = ["read_all", "profile:read_all", "activity:read_all"]
        redirect_url = "http://127.0.0.1:5000/authorization"

        # Creates an authorization URL using stravalib
        url = client.authorization_url(
            client_id=client_id,
            redirect_uri=redirect_url,
            scope=request_scope,
        )

        # Opens the URL and automatically gets the necessary code
        driver = webdriver.Firefox()
        driver.get(url)
        match = re.search(r"authorization?state=&code=([^&]+)", driver.current_url)
        while not match:
            time.sleep(0.5)
            match = re.search(r"code=([^&]+)", driver.current_url)
        code = match.group(1)
        driver.close()

        #Exchanges the code for an access token. Token is stored in json-file.
        client = Client()
        token_response = client.exchange_code_for_token(client_id=client_id, client_secret=client_secret, code=code)

        data = {'client_id': client_id,
                'client_secret': client_secret,
                'token': token_response
               }

        with open(json_path, "w") as file:
            json.dump(data, file, indent=4)

def getAuthenticatedClient(json_path = 'pw'):
    # Make sure the authentification is possible
    initialize_json(json_path)
    
    client = Client()
    with open(json_path, "r") as file:
            data = json.load(file)

    # Access values (client_id, client_secret, tokens)
    client_id = data.get("client_id")       
    client_secret = data.get("client_secret")  
    tokens = data.get("token")

    # Immediately refreshes token and stores it. New token is valid for 6 hours.
    token_response = client.refresh_access_token(
        client_id=client_id,
        client_secret=client_secret,
        refresh_token=tokens['refresh_token'])
    
    new_access_token = token_response["access_token"]
    
    data = {'client_id': client_id,
            'client_secret': client_secret,
            'token': token_response}
    
    with open(json_path, "w") as file:
        json.dump(data, file, indent=4)

    # Returns an authenticated client valid for 6 hours.
    return Client(access_token = new_access_token)

In [ ]:
#sets up an authenticated client. 
client = getAuthenticatedClient()

In [ ]:
#returns an iterator over the activities
client.get_activities()

In [ ]:
#gets information on the athlete
client.get_athlete()